In [9]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd

In [10]:
pothole = pd.read_csv("https://data.cityofchicago.org/resource/7as2-ds3y.csv?$limit=1000000")

In [11]:
patched = pd.read_csv("https://data.cityofchicago.org/resource/wqdh-9gek.csv?$limit=1000000")

In [12]:
pothole.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560478 entries, 0 to 560477
Data columns (total 19 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   creation_date                       560478 non-null  object 
 1   status                              560478 non-null  object 
 2   completion_date                     553130 non-null  object 
 3   service_request_number              560478 non-null  object 
 4   type_of_service_request             560478 non-null  object 
 5   current_activity                    393609 non-null  object 
 6   most_recent_action                  391992 non-null  object 
 7   number_of_potholes_filled_on_block  399143 non-null  float64
 8   street_address                      560476 non-null  object 
 9   zip                                 558160 non-null  float64
 10  x_coordinate                        560004 non-null  float64
 11  y_coordinate              

In [13]:
pothole.shape

(560478, 19)

In [14]:
pothole.columns

Index(['creation_date', 'status', 'completion_date', 'service_request_number',
       'type_of_service_request', 'current_activity', 'most_recent_action',
       'number_of_potholes_filled_on_block', 'street_address', 'zip',
       'x_coordinate', 'y_coordinate', 'ward', 'police_district',
       'community_area', 'ssa', 'latitude', 'longitude', 'location'],
      dtype='object')

In [15]:
patched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179958 entries, 0 to 179957
Data columns (total 7 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   address                             179958 non-null  object 
 1   request_date                        179958 non-null  object 
 2   completion_date                     179958 non-null  object 
 3   number_of_potholes_filled_on_block  179958 non-null  int64  
 4   latitude                            179489 non-null  float64
 5   longitude                           179489 non-null  float64
 6   location                            179489 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 9.6+ MB


In [16]:
patched.shape

(179958, 7)

In [17]:
patched.columns

Index(['address', 'request_date', 'completion_date',
       'number_of_potholes_filled_on_block', 'latitude', 'longitude',
       'location'],
      dtype='object')

In [18]:
pothole.sample(1)

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
378222,2013-08-26T00:00:00.000,Completed,2013-08-30T00:00:00.000,13-01219820,Pot Hole in Street,Final Outcome,Pothole Patched,4.0,401 E 107TH ST,60628.0,1.180818e+06,1.834097e+06,9.0,5.0,49.0,NaN,41.699902,-87.613508,"(41.699901866834, -87.613508152406)"


In [19]:
potholes_completed = pothole[pothole['status'] == 'Completed']

In [20]:
potholes_completed

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
42,2018-12-17T00:00:00.000,Completed,2018-12-18T00:00:00.000,18-03378095,Pothole in Street,Final Outcome,Pothole Patched,7.0,600 W LAKE ST,60661.0,1.172289e+06,1.901697e+06,42.0,12.0,28.0,NaN,41.885815,-87.642847,"(41.885814724037, -87.6428471339)"
58,2018-12-17T00:00:00.000,Completed,2018-12-18T00:00:00.000,18-03378491,Pothole in Street,Final Outcome,Completed Upon Arrival,0.0,719 N DRAKE AVE,60624.0,1.152585e+06,1.904606e+06,27.0,11.0,23.0,63.0,41.894335,-87.714912,"(41.894335117217, -87.714912418158)"
236,2018-12-15T00:00:00.000,Completed,2018-12-17T00:00:00.000,18-03366015,Pothole in Street,Final Outcome,Pothole Patched,4.0,230 N OGDEN AVE,60607.0,1.166961e+06,1.901783e+06,27.0,12.0,28.0,NaN,41.885849,-87.662604,"(41.885849434411, -87.662603973878)"
263,2018-12-15T00:00:00.000,Completed,2018-12-31T00:00:00.000,18-03365265,Pothole in Street,Final Outcome,Pothole Patched,17.0,8200 S SPRINGFIELD AVE,60652.0,1.151820e+06,1.849922e+06,18.0,8.0,70.0,NaN,41.744008,-87.719444,"(41.744007628888, -87.719443672392)"
289,2018-12-14T00:00:00.000,Completed,2018-12-14T00:00:00.000,18-03360709,Pothole in Street,Final Outcome,Pothole Patched,10.0,2844 W GRACE ST,60618.0,1.156639e+06,1.925124e+06,33.0,17.0,16.0,NaN,41.950429,-87.699970,"(41.950429023486, -87.6999704208)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560473,2010-03-11T00:00:00.000,Completed,2011-03-28T00:00:00.000,10-00317212,Pot Hole in Street,Dispatch Crew,Pothole Patched,5.0,7201 W BELMONT AVE,60634.0,NaN,NaN,36.0,25.0,17.0,NaN,41.937886,-87.806806,"(41.937885966284, -87.8068055064)"
560474,2010-02-26T00:00:00.000,Completed,2011-10-04T00:00:00.000,10-00254019,Pot Hole in Street,NaN,NaN,NaN,1440 S ROCKWELL ST,60608.0,1.159269e+06,1.892879e+06,28.0,10.0,29.0,NaN,41.861757,-87.690973,"(41.861756644625, -87.690972506101)"
560475,2009-08-27T00:00:00.000,Completed,2011-01-25T00:00:00.000,09-01501980,Pot Hole in Street,Site Survey,No Problem Found,80.0,5518 N WAYNE AVE,60640.0,NaN,NaN,48.0,20.0,77.0,NaN,41.982340,-87.663869,"(41.98234020574, -87.663868977471)"
560476,2009-06-17T00:00:00.000,Completed,2011-06-07T00:00:00.000,09-01026546,Pot Hole in Street,Dispatch Crew,Pothole Patched,NaN,3845 W ADDISON ST,60618.0,1.150061e+06,1.923655e+06,30.0,17.0,21.0,NaN,41.946303,-87.724348,"(41.946303283819, -87.724347984027)"


In [21]:
potholes_no_problem = pothole[pothole['most_recent_action'] == 'No Problem Found']

In [22]:
potholes_no_problem

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
445620,2012-08-24T00:00:00.000,Completed,2012-08-24T00:00:00.000,12-01509005,Pot Hole in Street,Final Outcome,No Problem Found,0.0,2300 N CLARK ST,60614.0,1.173105e+06,1.915576e+06,43.0,18.0,7.0,23.0,41.923816,-87.639555,"(41.923816495383, -87.639554930075)"
445714,2012-08-23T00:00:00.000,Completed,2012-08-24T00:00:00.000,12-01502801,Pot Hole in Street,Final Outcome,No Problem Found,0.0,155 E SUPERIOR ST,60611.0,1.177419e+06,1.905398e+06,42.0,18.0,8.0,NaN,41.895642,-87.623390,"(41.895641520773, -87.623389761865)"
445760,2012-08-23T00:00:00.000,Completed,2012-08-24T00:00:00.000,12-01501882,Pot Hole in Street,Final Outcome,No Problem Found,0.0,800 W 99TH ST,60643.0,1.172728e+06,1.839207e+06,21.0,22.0,73.0,45.0,41.714323,-87.643067,"(41.714323337011, -87.643066714219)"
445792,2012-08-22T00:00:00.000,Completed,2012-08-22T00:00:00.000,12-01494080,Pot Hole in Street,Final Outcome,No Problem Found,0.0,1727 S INDIANA AVE,60616.0,1.177932e+06,1.891926e+06,2.0,1.0,33.0,NaN,41.858299,-87.622194,"(41.858298638573, -87.622193507435)"
445908,2012-08-21T00:00:00.000,Completed,2012-08-21T00:00:00.000,12-01487704,Pot Hole in Street,Final Outcome,No Problem Found,0.0,2000 S WENTWORTH AVE,60616.0,1.175355e+06,1.890679e+06,25.0,9.0,34.0,NaN,41.855347,-87.631993,"(41.855346944832, -87.631992669955)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451625,2012-06-15T00:00:00.000,Completed,2012-07-18T00:00:00.000,12-01090125,Pot Hole in Street,Final Outcome,No Problem Found,0.0,800 W EASTMAN ST,60642.0,1.170751e+06,1.909618e+06,32.0,18.0,8.0,NaN,41.907582,-87.648253,"(41.907581773396, -87.648252683629)"
452125,2012-06-11T00:00:00.000,Completed,2012-07-17T00:00:00.000,12-01058012,Pot Hole in Street,Final Outcome,No Problem Found,0.0,2000 W 21ST ST,60608.0,1.163358e+06,1.890078e+06,25.0,12.0,31.0,NaN,41.854120,-87.676020,"(41.854119832467, -87.676019670842)"
452458,2012-06-07T00:00:00.000,Completed,2012-07-23T00:00:00.000,12-01040825,Pot Hole in Street,Final Outcome,No Problem Found,0.0,4301 S HOMAN AVE,60632.0,1.154397e+06,1.875926e+06,14.0,8.0,58.0,NaN,41.815293,-87.709014,"(41.815292549968, -87.709014351831)"
452978,2012-06-01T00:00:00.000,Completed,2012-07-17T00:00:00.000,12-01015272,Pot Hole in Street,Final Outcome,No Problem Found,0.0,2009 W 21ST ST,60608.0,1.163298e+06,1.890076e+06,25.0,12.0,31.0,NaN,41.853896,-87.676318,"(41.85389629901, -87.676318396234)"


In [121]:
sample_pothole = pothole.sample(5)

In [122]:
sample_pothole

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
263205,2014-06-24T00:00:00.000,Completed - Dup,2014-08-11T00:00:00.000,14-00977124,Pothole in Street,NaN,NaN,NaN,1251 W 31ST ST,60608.0,1.168424e+06,1.884243e+06,11.0,9.0,60.0,NaN,41.837775,-87.658050,"(41.837774884813, -87.658049660097)"
295408,2014-04-13T00:00:00.000,Completed,2014-04-13T00:00:00.000,14-00533196,Pothole in Street,Final Outcome,Pothole Patched,31.0,5000 N CENTRAL PARK AVE,60625.0,1.151483e+06,1.932984e+06,39.0,17.0,14.0,NaN,41.972052,-87.718503,"(41.97205196209, -87.718503207688)"
471579,2012-02-21T00:00:00.000,Completed,2012-02-21T00:00:00.000,12-00287174,Pot Hole in Street,Dispatch Crew,Pothole Patched,7.0,1620 W 115TH ST,60643.0,1.167452e+06,1.828447e+06,34.0,22.0,75.0,NaN,41.684913,-87.662620,"(41.684913490319, -87.66262038721)"
285987,2014-04-30T00:00:00.000,Completed - Dup,2014-10-15T00:00:00.000,14-00628561,Pothole in Street,NaN,NaN,NaN,7400 S EXCHANGE AVE,60649.0,1.194717e+06,1.856393e+06,7.0,3.0,43.0,49.0,41.760710,-87.561946,"(41.760710194223, -87.561946030143)"
537499,2011-03-10T00:00:00.000,Completed - Dup,2011-03-10T00:00:00.000,11-00395768,Pot Hole in Street,NaN,NaN,NaN,6609 N CLARK ST,60626.0,1.164018e+06,1.944001e+06,40.0,24.0,1.0,24.0,42.002027,-87.671819,"(42.002027180902, -87.671819018048)"


In [123]:
for column in sample_pothole.iterrows():
  if column[1]['status'] == 'Completed':
    print('its completed!')
  elif column[1]['status'] == 'Completed - Dup':
    print('its completed!')
  else:
    print('it hasnt been completed')

its completed!
its completed!
its completed!
its completed!
its completed!
